In [16]:
from azureml.core.authentication import AzureCliAuthentication
import azureml.core
from azureml.core import Workspace

cli_auth = AzureCliAuthentication()
ws = Workspace(subscription_id='401a2f63-c029-433e-9087-c839197089fd', resource_group='pix2para_resourcegroup', workspace_name = 'pix2para', auth=cli_auth)

In [17]:
experiment_name = 'vgg_test'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)

In [37]:
ws.list(subscription_id = '401a2f63-c029-433e-9087-c839197089fd')

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


{'pix2para': [Workspace.create(name='pix2para', subscription_id='401a2f63-c029-433e-9087-c839197089fd', resource_group='pix2para_resourcegroup')]}

In [18]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "cpu-cluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print("found compute target: " + compute_name)
else:
    print("creating new compute target...")
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
     # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

found compute target: cpu-cluster


In [8]:
import os
script_folder = os.path.join(os.getcwd(), "vggtest")
os.makedirs(script_folder, exist_ok=True)

In [13]:
from keras.applications.vgg19 import VGG19
import joblib
os.makedirs('outputs', exist_ok=True)
model = VGG19()
# note file saved in the outputs folder is automatically uploaded into experiment record
joblib.dump(value=model, filename='outputs/vgg.pkl')

['outputs/vgg.pkl']

In [24]:
from keras.applications.vgg19 import VGG19

model = VGG19()

In [66]:
%%writefile score.py
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg19 import decode_predictions
from keras.applications.vgg19 import VGG19
import numpy as np
import io
from PIL import Image
import json

def init():
    global model
    model = VGG19()

def run(image_bytes):
    #data = np.array(image)
    # make prediction
    image_bytes = json.loads(image_bytes)['data']
    image = Image.open(io.BytesIO(image_bytes))
    image = image.resize((224,224),Image.ANTIALIAS)
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    yhat = model.predict(image)
    label = decode_predictions(yhat)
    label = label[0][0]
    return label


Overwriting score.py


In [47]:
%%writefile score.py

import json 
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg19 import decode_predictions
from keras.applications.vgg19 import VGG19
import numpy as np
import io
from PIL import Image
import json
from numpy import array

def run(image_bytes):

    image_bytes = json.loads(image_bytes)['data'][0]
    image_bytes = image_bytes.encode('utf-8')
    encode_len = len(image_bytes)
    print(encode_len)
    
    image = Image.frombytes('RGBA', (1315,640), image_bytes, 'raw')
    
    image = image.resize((224,224),Image.ANTIALIAS)    
    image = img_to_array(image)
    image = image[:,:,0:3]
    print(image.shape)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    
    model = VGG19()
    yhat = model.predict(image)
    label = decode_predictions(yhat)
    label = label[0][0]
    return label

Overwriting score.py


In [11]:
import base64
import json 
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg19 import decode_predictions
from keras.applications.vgg19 import VGG19
import numpy as np
import io
from PIL import Image
import json
from numpy import array

def run(image_bytes):
    #data = np.array(image)
    # make prediction
    image_bytes = json.loads(image_bytes)['data'][0]
    #print(image_bytes)
    image_bytes = image_bytes.encode('utf-8')
    encode_len = len(image_bytes)
    print(encode_len)
    
    image = Image.frombytes('RGBA', (1315,640), image_bytes, 'raw')
    
    image = image.resize((224,224),Image.ANTIALIAS)
    pilimg = list(Image.open(image_path, 'r').getdata())
    pix_val_flat = [x for sets in pilimg for x in sets]
    print(pix_val_flat[0:10])
    image.show()
    
    image = img_to_array(image)
    image = image[:,:,0:3]
    print(image.shape)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    
    model = VGG19()
    yhat = model.predict(image)
    label = decode_predictions(yhat)
    label = label[0][0]
    return label

image_path = '/Users/audreycui01/New Unity Project/assets/screenshot.png'
image = Image.open(image_path, 'r')
#image.show()
pilimg = list(image.getdata())
#pix_val_flat = [x for sets in pilimg for x in sets]
print(pilimg[0:100])
print(array(pilimg).shape)
#image_bytes = base64.b64encode(pix_val_flat)
image_bytes = bytearray(pix_val_flat)
image_bytes = str(image_bytes)
#image_bytes = image_bytes.decode('utf-8')



'''
with open(image_path, "rb") as imageFile:
    image_bytes = base64.b64encode(imageFile.read())
    image_bytes = image_bytes.decode('utf-8')
'''

decode_len = len(image_bytes)
print(decode_len)
data = {"data": [image_bytes]}
input_data = json.dumps(data)
#print(input_data)
label = run(input_data)
print(label)

[(130, 130, 130, 255), (130, 130, 130, 255), (104, 104, 104, 255), (104, 104, 104, 255), (112, 116, 106, 255), (112, 116, 106, 255), (90, 94, 82, 255), (90, 95, 82, 255), (90, 95, 82, 255), (90, 95, 82, 255), (89, 94, 81, 255), (89, 94, 81, 255), (88, 92, 79, 255), (87, 91, 78, 255), (87, 91, 78, 255), (86, 90, 77, 255), (86, 88, 76, 255), (85, 87, 75, 255), (85, 87, 75, 255), (85, 87, 75, 255), (88, 89, 77, 255), (90, 90, 78, 255), (80, 79, 68, 255), (70, 69, 57, 255), (52, 51, 42, 255), (35, 32, 26, 255), (32, 29, 23, 255), (31, 27, 21, 255), (36, 32, 27, 255), (42, 37, 34, 255), (47, 42, 39, 255), (51, 47, 43, 255), (51, 47, 43, 255), (49, 47, 41, 255), (47, 46, 40, 255), (46, 45, 38, 255), (46, 45, 38, 255), (45, 45, 38, 255), (44, 45, 38, 255), (44, 44, 37, 255), (44, 44, 37, 255), (44, 45, 38, 255), (44, 45, 38, 255), (43, 44, 37, 255), (43, 44, 37, 255), (44, 45, 38, 255), (44, 45, 38, 255), (45, 46, 39, 255), (44, 46, 38, 255), (44, 45, 37, 255), (43, 45, 35, 255), (42, 44, 35,

In [ ]:
print ("hello world")

In [22]:
from azureml.core.model import Model

model_name = "vggtest"


In [31]:
import os
model = Model.register(model_path="outputs/vgg.pkl",
                        model_name=model_name,
                        description="vgg test",
                        workspace=ws)
#model.download(target_dir=os.getcwd(), exist_ok=True)

Registering model vggtest


In [65]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_pip_package("scikit-learn==0.22.1")
myenv.add_pip_package("keras.applications")
myenv.add_pip_package("keras.preprocessing")
myenv.add_pip_package("azureml-defaults")
myenv.add_pip_package("keras")
myenv.add_pip_package("tensorflow")
myenv.add_pip_package("numpy")
myenv.add_pip_package("pillow")


with open("my_vgg_env.yml","w") as f:
    f.write(myenv.serialize_to_string())

In [26]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=8, 
                                               description='vggtest')

In [ ]:
%%time
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment


myenv = Environment.from_conda_specification(name="my_vgg_env", file_path="my_vgg_env.yml")
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

service = Model.deploy(workspace=ws, 
                       name='vggtest', 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aciconfig)

service.wait_for_deployment(show_output=True)

Running.............................................................................

In [33]:
print(service.get_logs())

/bin/bash: /azureml-envs/azureml_324f55ab119cb8c9bb8f3505406b38c5/lib/libtinfo.so.5: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_324f55ab119cb8c9bb8f3505406b38c5/lib/libtinfo.so.5: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_324f55ab119cb8c9bb8f3505406b38c5/lib/libtinfo.so.5: no version information available (required by /bin/bash)
2020-03-08T19:31:00,991622920+00:00 - nginx/run 
2020-03-08T19:31:00,991706421+00:00 - rsyslog/run 
/bin/bash: /azureml-envs/azureml_324f55ab119cb8c9bb8f3505406b38c5/lib/libtinfo.so.5: no version information available (required by /bin/bash)
/usr/sbin/nginx: /azureml-envs/azureml_324f55ab119cb8c9bb8f3505406b38c5/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_324f55ab119cb8c9bb8f3505406b38c5/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/ngin

In [49]:
service.delete()

http://44ebd92f-f3e4-4524-a50a-b3569a653cd2.westus.azurecontainer.io/score
Healthy


In [48]:
service = AciWebservice(ws, 'vggtest')

inference_config = InferenceConfig(entry_script="score.py", environment=myenv)
myenv = Environment.from_conda_specification(name="my_vgg_env", file_path="my_vgg_env.yml")

service.update(inference_config = inference_config)

Received bad response from Model Management Service:
Response Code: 409
Headers: {'Date': 'Mon, 09 Mar 2020 03:16:41 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Request-Context': 'appId=cid-v1:2d2e8e63-272e-4b3c-8598-4ee570a0e70d', 'x-ms-client-request-id': '459980a12afe41fa90aa43117a4a5f25', 'x-ms-client-session-id': '', 'api-supported-versions': '1.0, 2018-03-01-preview, 2018-11-19', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload'}
Content: b'{"code":"Conflict","statusCode":409,"message":"Conflict","details":[{"code":"ConflictOfOperation","message":"Conflict of operation, another operation on same entity is already running"}]}'



WebserviceException: WebserviceException:
	Message: Received bad response from Model Management Service:
Response Code: 409
Headers: {'Date': 'Mon, 09 Mar 2020 03:16:41 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Request-Context': 'appId=cid-v1:2d2e8e63-272e-4b3c-8598-4ee570a0e70d', 'x-ms-client-request-id': '459980a12afe41fa90aa43117a4a5f25', 'x-ms-client-session-id': '', 'api-supported-versions': '1.0, 2018-03-01-preview, 2018-11-19', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload'}
Content: b'{"code":"Conflict","statusCode":409,"message":"Conflict","details":[{"code":"ConflictOfOperation","message":"Conflict of operation, another operation on same entity is already running"}]}'
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Received bad response from Model Management Service:\nResponse Code: 409\nHeaders: {'Date': 'Mon, 09 Mar 2020 03:16:41 GMT', 'Content-Type': 'applica

In [ ]:
print(service.scoring_uri)
print(service.state)
